In [ ]:
!sudo apt install tesseract-ocr && pip install theos-ai[ocr]==0.0.28

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 22 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 1s (6,077 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fro

In [ ]:
!theos setup --version v1 

[theos] please, select one of the following AI subfields to setup:

1. Object Detection
2. OCR

[*] enter your choice (1 or 2): 1

[theos] setting up object-detection subfield...

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scipy-1.4.1.tar.gz (24.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached torch-1.11.0-cp39-cp39-manylinux1_x86_64.whl (750.6 MB)
  Using cached torchvision-0.12.0-cp39-cp39-manylinux1_x86_64.whl (21.0 MB)
  Using cached seaborn-0.11.0-py3-none-any.whl (283 kB)
  Using cached pandas-1.3.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
  Using cached tensorflow-2.10.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (578.1 MB)
  Using cached tensorflowjs-4.0.0-py3-none-any.whl (83 kB)
  Using cached tensorflow_decision_forests-1.0.1-cp39-cp39-manylinux_2_17_x86_64.man

In [ ]:
!theos login


 Theos CLI

[*] username: vishvasena_22
[*] password: 

[theos] login succeeded


In [ ]:
!theos set-project 4f9a51f3-a119-4ec4-9076-c9d682784fb6

[theos] project added successfully


In [ ]:
!theos install yolov7 --version tiny --weights lisence-plate:experiment-1:best

[theos] downloading yolov7 code:

100% 272k/272k [00:00<00:00, 1.93MB/s]

[theos] extracting yolov7 code...
[theos] downloading lisence-plate weights for the tiny version of yolov7:

100% 11.7M/11.7M [00:01<00:00, 10.6MB/s]

[theos] lisence-plate experiment-1:best weights downloaded successfully ✓


In [ ]:
ALGORITHM = 'yolov7' 
ALGORITHM_VERSION = 'tiny' 
WEIGHTS = 'lisence-plate:experiment-1:best' 

OCR_MODEL_SIZE = 'large' 
OCR_MODEL_TYPE = 'printed' 
OCR_MODEL_ACCURACY = 'best' 
OCR_CLASSES = ['lisenceplate labeled'] 

INPUT_VIDEO = '/content/0.5 speed.mp4'
OUTPUT_VIDEO = 'output.mp4'

In [ ]:
from theos.computer_vision.object_detection.utils import draw
from theos.computer_vision import ocr
from theos.client import Client
from google.colab import files
from tqdm import tqdm
import torch
import json
import time
import cv2
import os

if torch.cuda.is_available():
  torch.cuda.empty_cache()

client = Client(inputs='.', outputs='.')
yolov7 = client.get(ALGORITHM, version=ALGORITHM_VERSION)
yolov7.load_weights(WEIGHTS)
yolov7.to_gpu()

if OCR_MODEL_TYPE and OCR_MODEL_ACCURACY: 
  ocr_model = ocr.load(size=OCR_MODEL_SIZE, model_type=OCR_MODEL_TYPE, accuracy=OCR_MODEL_ACCURACY)
else:
  ocr_model = ocr.load(size=OCR_MODEL_SIZE)

if OCR_MODEL_SIZE == 'large':
  ocr_model.to_gpu()

video = cv2.VideoCapture(INPUT_VIDEO)
width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
frames_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
output = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (width, height))

if video.isOpened() == False:
	print('[!] error opening the video')

print('[theos] detecting video...\n')
pbar = tqdm(total=frames_count, unit=' frames', dynamic_ncols=True, position=0, leave=True)

while video.isOpened():
    ret, frame = video.read()
    if ret == True:
        detections = yolov7.detect(frame)
        detections = ocr_model.read(frame, detections=detections, classes=OCR_CLASSES)
        detected_frame = draw(frame, detections, alpha=0.15)
        output.write(detected_frame)
        pbar.update(1)
    else:
        break

pbar.close()
print('\n')
video.release()
output.release()
yolov7.unload()
client.close()

print('[theos] downloading output video...\n')
files.download(OUTPUT_VIDEO)

[theos] importing yolov7 code...
[theos] yolov7 code imported successfully
[theos] loading yolov7...


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system



[theos] loading the large OCR model...



[theos] large OCR model loaded successfully
[theos] detecting video...



100%|██████████| 938/938 [01:03<00:00, 14.72 frames/s]




[theos] unloading instance of yolov7...
[theos] done
[theos] downloading output video...



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>